# Metin Temsili Yöntemleri
# Transformers Tabanlı Metin Temsili

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

"bert-base-uncased" modeli, Google tarafından geliştirilen ve Transformers mimarisine dayanan BERT (Bidirectional Encoder Representations from Transformers) modelinin en yaygın kullanılan versiyonlarından biridir.

| Özellik                             | Açıklama                                                             |
| ----------------------------------- | -------------------------------------------------------------------- |
| **Model Adı**                       | `bert-base-uncased`                                                  |
| **Katman Sayısı (layers)**          | 12                                                                   |
| **Gizli Boyut (hidden size)**       | 768                                                                  |
| **Başlık Sayısı (attention heads)** | 12                                                                   |
| **Toplam Parametre**                | 110 milyon                                                           |
| **Tokenizasyon**                    | WordPiece                                                            |
| **Uncased**                         | Tüm harfler küçük harfe çevrilmiş; büyük-küçük harf ayrımı yapılmaz. |


wordpiece: kelimeleri daha küçük parçalara (subword units) bölen bir tokenizasyon yöntemidir.

🧠 BERT'in Ana Fikri:

* Bidirectional (çift yönlü): Bir kelimenin anlamını önceki ve sonraki kelimelere bakarak anlar.

* Pre-trained: Büyük metin koleksiyonları (Wikipedia, BookCorpus) üzerinde önceden eğitilmiştir.

* Fine-tune: Belirli bir NLP görevi (sınıflandırma, soru yanıtlama, NER vs.) için sonradan eğitilebilir.

In [ ]:
# BERT modelini kullanacağız:

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# input metin tanımlayalım:

text = "Transformers can be used for language processing"

In [ ]:
# metni tokenlara çevirelim:

inputs = tokenizer(text, return_tensors="pt") #çıktı pytorch tensoru olarak return edilir

In [ ]:
inputs.keys()

# dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids : Metnin tokenizer tarafından parçalanıp sayısal ID’lere dönüştürülmüş halidir.

Her kelime ya da alt kelime (subword) bir sayıya karşılık gelir.

token_type_ids :Segment embeddings olarak da bilinir.

BERT, iki cümle arasındaki ilişkileri modellemek için eğitildiği için her token’ın hangi cümleye ait olduğunu belirtmek için kullanılır:

0 → 1. cümle

1 → 2. cümle

attention_mask : Modelin hangi token’lara dikkat etmesi gerektiğini belirtir.

1 → dikkate alınacak token

0 → padding (boşluk), yok sayılır

In [ ]:
# modeli kullanarak metni vektör ile temsil edelim:
with torch.no_grad(): #belleği verimli kullanmak için gradyanların hesaplanması durdurulur
    outputs = model(**inputs)


In [ ]:
# modelin çıktısından son gizli durumu alalım:
last_hidden_state = outputs.last_hidden_state  # tüm token çıktılarını almak için
last_hidden_state # -> her bir token'ın bağlamlı anlamını taşıyan vektörler.

tensor([[[-0.2226, -0.0053,  0.0619,  ..., -0.6325, -0.5102,  0.2819],
         [ 1.6193, -0.6504, -0.4584,  ..., -0.1557, -0.2971,  0.1700],
         [ 0.3051,  0.3551,  0.1914,  ..., -0.7976, -0.2410, -0.3585],
         ...,
         [-0.4816, -0.4254,  0.3245,  ..., -0.1562, -0.7008,  0.1906],
         [-0.0398, -0.2221, -0.2973,  ..., -0.5196, -0.7218, -0.3353],
         [ 0.9008, -0.0643, -0.6290,  ..., -0.0408, -0.5067, -0.1396]]])

In [ ]:
outputs.last_hidden_state.shape  # torch.Size([batch_size, sequence_length, hidden_size])

torch.Size([1, 9, 768])

1 → batch size (tek bir örnek)

9 → token sayısı

768 → BERT-base modelinde her token için üretilen vektör boyutu

In [ ]:
# cümle 7 kelime olmasına rağmen neden 9 token çıkıyor?

print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))  # özel tokenlarla birlikte

['[CLS]', 'transformers', 'can', 'be', 'used', 'for', 'language', 'processing', '[SEP]']


# BERT modelinde Özel Tokenlar (Special Tokens):

* `[CLS]`   **Classification token** – Her giriş cümlesinin başına eklenir. <br>Model bu token’ın çıktısını (vektörünü) tüm cümlenin temsili olarak kullanır.


*  `[SEP]`   **Separator token** – Cümleleri ayırmak için kullanılır. <br>Tek cümlede de sonuna eklenir. İki cümle varsa araya ve sona konur.

* `[PAD]`   **Padding token** – Cümleler eşit uzunlukta olsun diye kısa olanlara eklenir.                                                                     
* `[MASK]`  **Masking token** – Masked Language Model (BERT’in eğitildiği görev) için belirli kelimeleri gizlemek amacıyla kullanılır.                        


In [ ]:
# ilk tokeni (CLS) embedding için alalım:

first_token_embedding = last_hidden_state[0, 0, :].numpy()
metin_temsili = first_token_embedding
metin_temsili

array([-2.22624645e-01, -5.30474307e-03,  6.19363114e-02, -3.18443924e-02,
       -4.61473852e-01, -6.32674014e-03,  3.83154035e-01,  3.37532878e-01,
       -1.39316887e-01,  1.66419949e-02, -1.11417398e-01, -2.84839064e-01,
        1.13537073e-01,  3.94788414e-01, -2.23026738e-01, -4.48534042e-02,
       -3.83132458e-01,  5.94088376e-01,  1.96287856e-01,  1.18655123e-01,
       -2.05818027e-01, -6.89052522e-01,  5.43743014e-01,  1.15593426e-01,
       -2.74032384e-01, -3.42320859e-01,  1.31592780e-01,  9.86589305e-03,
       -1.98698893e-01,  1.56432018e-02, -2.71932036e-01,  4.78954613e-01,
       -2.95576975e-02, -1.79637849e-01,  6.74694598e-01,  5.65526895e-02,
        1.49323329e-01, -3.89797628e-01, -4.18173475e-03,  8.67525786e-02,
        7.95310438e-02, -5.92655763e-02,  4.07361805e-01,  8.47081169e-02,
       -2.73247540e-01,  4.34541367e-02, -2.21283340e+00, -1.42286956e-01,
       -3.71601522e-01, -3.92912865e-01, -1.51401579e-01,  2.19305575e-01,
        9.07798260e-02,  

NOT: BERT gibi modeller, [CLS] tokenı için özel bir yer tutar.

Model, tüm cümlenin bilgisini buraya toplamaya çalışır. CLS tokeni, cümlenin genel anlamını yansıtır şekilde optimize edilir.